# Exchanging Solutions Between Two Nonmatching Spherical Surfaces

In [1]:
import numpy as np
from pydtk2 import *
from pymoab.core import Core
from pymoab.types import MBQUAD, MB_TYPE_DOUBLE, MB_TAG_DENSE, MBVERTEX
from mpi4py import MPI
from math import sqrt, sin, cos, pi, exp
import scipy.io as sio

In [2]:
choose_level = 4
assert choose_level >= 0 and choose_level <= 4 and isinstance(choose_level, int)
use_wu = True

# the next line should be modified for maximum iterations
itmax = 200
radius_scale = 3.0

if use_wu:
    bo = WU4
else:
    bo = WENLAND4

matoutput1 = 'CubeSph%i.mat' % choose_level
matoutput2 = 'Unstr%i.mat' % choose_level


levels = []
mesh_root = 'meshes/untri/'
levels.append(
    (mesh_root+'CubSph_0_0_0_26_31_43_GaussHills.vtk',
     mesh_root+'sphere0.vtk')
)
levels.append(
    (mesh_root+'CubSph_0_0_0_36_43_60_GaussHills.vtk',
     mesh_root+'sphere1.vtk')
)
levels.append(
    (mesh_root+'CubSph_0_0_0_50_60_84_GaussHills.vtk',
     mesh_root+'sphere2.vtk')
)
levels.append(
    (mesh_root+'CubSph_0_0_0_70_84_118_GaussHills.vtk',
     mesh_root+'sphere3.vtk')
)

def f1(x):
    return (sin(x[0])+cos(x[1]))*x[2]

def f2(x):
    return (0.25*x[0]**2+0.33333333333333*x[1]**2+0.5*x[2]**2)*(x[0]+x[1]+x[2])

gauss1 = np.array([sin(0.5*pi)*cos(5*pi/6), sin(0.5*pi)*sin(5*pi/6), cos(0.5*pi)])
gauss2 = np.array([sin(0.5*pi)*cos(7*pi/6), sin(0.5*pi)*sin(7*pi/6), cos(0.5*pi)])
def fgauss(x):
    g1 = x-gauss1
    g2 = x-gauss2
    return 0.95*(
        exp(-5.0*np.dot(g1, g1))+exp(-5.0*np.dot(g2,g2))
    )

In [3]:
if not choose_level:
    coarse_file = 'meshes/CubSph2_0_0_0_41_55_63_GaussHillSlotCylnd.vtk'
    fine_file = 'meshes/CubSph2_0_0_0_73_81_101_GaussHillSlotCylnd.vtk'
else:
    coarse_file = levels[choose_level-1][1]
    fine_file = levels[choose_level-1][0]

In [4]:

# the next line should be modified for output file names
output_common = 'results/DTKresultsLevel%i' % choose_level
output_fine, output_coarse = output_common + '_fine', output_common + '_coarse'

In [5]:
# MOAB stuffs
mdb_fine, mdb_coarse = Core(), Core()

# load files, this will store point_data in tags
mdb_fine.load_file(fine_file)
mdb_coarse.load_file(coarse_file)


# the point data in vtk files are tagged with 'scalars'
field_fine = field_coarse = 'scalars'
ftag = mdb_fine.tag_get_handle(field_fine)
if choose_level:
    verts = mdb_coarse.get_entities_by_type(0, MBVERTEX)
    ctag = mdb_coarse.tag_get_handle(
        field_coarse,
        size=1,
        tag_type=MB_TYPE_DOUBLE,
        storage_type=MB_TAG_DENSE,
        create_if_missing=True)
    dummy = np.zeros(verts.size(), dtype=np.float64)
    mdb_coarse.tag_set_data(ctag, verts, dummy)

verts1 = mdb_fine.get_entities_by_type(0, MBVERTEX)
tagff1 = mdb_fine.tag_get_handle(
        'f1',
        size=1,
        tag_type=MB_TYPE_DOUBLE,
        storage_type=MB_TAG_DENSE,
        create_if_missing=True)
tagff2 = mdb_fine.tag_get_handle(
        'f2',
        size=1,
        tag_type=MB_TYPE_DOUBLE,
        storage_type=MB_TAG_DENSE,
        create_if_missing=True)
tagff1err = mdb_fine.tag_get_handle(
        'f1Error',
        size=1,
        tag_type=MB_TYPE_DOUBLE,
        storage_type=MB_TAG_DENSE,
        create_if_missing=True)
tagff2err = mdb_fine.tag_get_handle(
        'f2Error',
        size=1,
        tag_type=MB_TYPE_DOUBLE,
        storage_type=MB_TAG_DENSE,
        create_if_missing=True)
tagfgerr = mdb_fine.tag_get_handle(
        'GaussError',
        size=1,
        tag_type=MB_TYPE_DOUBLE,
        storage_type=MB_TAG_DENSE,
        create_if_missing=True)
ff1 = np.zeros(verts1.size(), dtype=np.float64)
ff2 = np.zeros(verts1.size(), dtype=np.float64)
fg = np.zeros(verts1.size(), dtype=np.float64)

verts2 = mdb_coarse.get_entities_by_type(0, MBVERTEX)
tagcf1 = mdb_coarse.tag_get_handle(
        'f1',
        size=1,
        tag_type=MB_TYPE_DOUBLE,
        storage_type=MB_TAG_DENSE,
        create_if_missing=True)
tagcf2 = mdb_coarse.tag_get_handle(
        'f2',
        size=1,
        tag_type=MB_TYPE_DOUBLE,
        storage_type=MB_TAG_DENSE,
        create_if_missing=True)
tagcf1err = mdb_coarse.tag_get_handle(
        'f1Error',
        size=1,
        tag_type=MB_TYPE_DOUBLE,
        storage_type=MB_TAG_DENSE,
        create_if_missing=True)
tagcf2err = mdb_coarse.tag_get_handle(
        'f2Error',
        size=1,
        tag_type=MB_TYPE_DOUBLE,
        storage_type=MB_TAG_DENSE,
        create_if_missing=True)
tagcgerr = mdb_coarse.tag_get_handle(
        'GaussError',
        size=1,
        tag_type=MB_TYPE_DOUBLE,
        storage_type=MB_TAG_DENSE,
        create_if_missing=True)
cf1 = np.zeros(verts2.size(), dtype=np.float64)
cf2 = np.zeros(verts2.size(), dtype=np.float64)
cg = np.zeros(verts2.size(), dtype=np.float64)
    
# scan through the average edge lengths for two meshes
fine_coords = mdb_fine.get_coords(mdb_fine.get_entities_by_dimension(0,0)).reshape(verts1.size(), 3)
coarse_coords = mdb_coarse.get_coords(mdb_coarse.get_entities_by_dimension(0,0)).reshape(verts2.size(), 3)

# get element
# we know fine is quad, coarse is tri
quads = mdb_fine.get_entities_by_dimension(0, 2)
tris = mdb_coarse.get_entities_by_dimension(0, 2)

fine_h = coarse_h = -1.0

def dist(x):
    return sqrt(x[0]**2+x[1]**2+x[2]**2)

for i in range(quads.size()):
    conn = mdb_fine.get_connectivity(quads[i])
    conn[:] -= 1
    ds = fine_coords[conn]
    dx = ds[3]-ds[0]
    fine_h = max(dist(dx), fine_h)
    for j in range(3):
        dx = ds[j+1]-ds[j]
        fine_h = max(fine_h, dist(dx))
        
for i in range(tris.size()):
    conn = mdb_coarse.get_connectivity(tris[i])
    conn[:] -= 1
    ds = coarse_coords[conn]
    dx = ds[2] - ds[0]
    coarse_h = max(dist(dx), coarse_h)
    for j in range(2):
        dx = ds[j+1]-ds[j]
        coarse_h = max(dist(dx), coarse_h)

min_x = min_y = min_z = 1e30
max_x = max_y = max_z = 1e-30

for i in range(fine_coords.shape[0]):
    ff1[i] = f1(fine_coords[i])
    ff2[i] = f2(fine_coords[i])
    min_x = min(fine_coords[i][0], min_x)
    max_x = max(fine_coords[i][0], max_x)
    min_y = min(fine_coords[i][1], min_y)
    max_y = max(fine_coords[i][1], max_y)
    min_z = min(fine_coords[i][2], min_z)
    max_z = max(fine_coords[i][2], max_z)

largest = max(max(abs(max_z-min_z), abs(max_y-min_y)), abs(max_x-min_x))
min_x = min_y = min_z = 1e30
max_x = max_y = max_z = 1e-30
# if fine_h*radius_scale>=0.2*largest:
#     fine_h = 0.2*largest/radius_scale
    
for i in range(coarse_coords.shape[0]):
    cf1[i] = f1(coarse_coords[i])
    cf2[i] = f2(coarse_coords[i])
    min_x = min(coarse_coords[i][0], min_x)
    max_x = max(coarse_coords[i][0], max_x)
    min_y = min(coarse_coords[i][1], min_y)
    max_y = max(coarse_coords[i][1], max_y)
    min_z = min(coarse_coords[i][2], min_z)
    max_z = max(coarse_coords[i][2], max_z)

largest = max(max(abs(max_z-min_z), abs(max_y-min_y)), abs(max_x-min_x))
# if coarse_h*radius_scale>=0.2*largest:
#     coarse_h = 0.2*largest/radius_scale
    
mdb_fine.tag_set_data(tagff1, verts1, ff1)
mdb_fine.tag_set_data(tagff2, verts1, ff2)
mdb_coarse.tag_set_data(tagcf1, verts2, cf1)
mdb_coarse.tag_set_data(tagcf2, verts2, cf2)

In [6]:
# Now construct two PyDTK2 MOAB managers
fine_manager = DTK2MoabManager(mdb_fine)
coarse_manager = DTK2MoabManager(mdb_coarse)

In [7]:
# now we push the tag strings into the managers
fine_manager.register_tag(field_fine)
fine_manager.register_tag('f1')
fine_manager.register_tag('f2')
coarse_manager.register_tag(field_coarse)
coarse_manager.register_tag('f1')
coarse_manager.register_tag('f2')

In [8]:
print('fine->coarse radius: %f.' % (fine_h*radius_scale))
print('coarse->fine radius: %f.' % (coarse_h*radius_scale))

fine->coarse radius: 0.086944.
coarse->fine radius: 0.208786.


In [9]:
# next step is to construct the DTK2 MOAB mappers
fine2coarse = DTK2Mapper(
    source=fine_manager,
    target=coarse_manager,
    r=fine_h*radius_scale,
    basis_order=bo
)
coarse2fine = DTK2Mapper(
    source=coarse_manager,
    target=fine_manager,
    r=coarse_h*radius_scale,
    basis_order=bo
)

# now register coupled flags
fine2coarse.register_coupled_tags(field_fine, field_coarse)
fine2coarse.register_coupled_tags('f1', 'f1')
fine2coarse.register_coupled_tags('f2', 'f2')
coarse2fine.register_coupled_tags(field_coarse, field_fine)
coarse2fine.register_coupled_tags('f1', 'f1')
coarse2fine.register_coupled_tags('f2', 'f2')

In [10]:
fgerrL2 = np.zeros(itmax)
ff1L2 = np.zeros(itmax)
ff2L2 = np.zeros(itmax)
fgerrInf = np.zeros(itmax)
ff1Inf = np.zeros(itmax)
ff2Inf = np.zeros(itmax)

cgerrL2 = np.zeros(itmax)
cf1L2 = np.zeros(itmax)
cf2L2 = np.zeros(itmax)
cgerrInf = np.zeros(itmax)
cf1Inf = np.zeros(itmax)
cf2Inf = np.zeros(itmax)

In [11]:
# the transfer loop
for it in range(itmax):
    # transfer from fine to coarse
    fine2coarse.apply(field_fine, field_coarse)
    fine2coarse.apply('f1', 'f1')
    fine2coarse.apply('f2', 'f2')
    
    # retrieve values from coarse
    cg[:] = mdb_coarse.tag_get_data(ctag, verts2, flat=True)
    cf1[:] = mdb_coarse.tag_get_data(tagcf1, verts2, flat=True)
    cf2[:] = mdb_coarse.tag_get_data(tagcf2, verts2, flat=True)
    for i in range(verts2.size()):
        cg[i] -= fgauss(coarse_coords[i])
        cf1[i] -= f1(coarse_coords[i])
        cf2[i] -= f2(coarse_coords[i])
    
    mdb_coarse.tag_set_data(tagcf1err, verts2, cf1)
    mdb_coarse.tag_set_data(tagcf2err, verts2, cf2)
    mdb_coarse.tag_set_data(tagcgerr, verts2, cg)
    
    cgerrL2[it] = sqrt(np.dot(cg, cg)/verts2.size())
    cf1L2[it] = sqrt(np.dot(cf1, cf1)/verts2.size())
    cf2L2[it] = sqrt(np.dot(cf2, cf2)/verts2.size())
    cgerrInf[it] = max(abs(cg))
    cf1Inf[it] = max(abs(cf1))
    cf2Inf[it] = max(abs(cf2))
    
    # transfer from coarse to fine
    coarse2fine.apply(field_coarse, field_fine)
    coarse2fine.apply('f1', 'f1')
    coarse2fine.apply('f2', 'f2')
    
    fg[:] = mdb_fine.tag_get_data(ftag, verts1, flat=True)
    ff1[:] = mdb_fine.tag_get_data(tagff1, verts1, flat=True)
    ff2[:] = mdb_fine.tag_get_data(tagff2, verts1, flat=True)
    for i in range(verts1.size()):
        fg[i] -= fgauss(fine_coords[i])
        ff1[i] -= f1(fine_coords[i])
        ff2[i] -= f2(fine_coords[i])
    
    mdb_fine.tag_set_data(tagff1err, verts1, ff1)
    mdb_fine.tag_set_data(tagff2err, verts1, ff2)
    mdb_fine.tag_set_data(tagfgerr, verts1, fg)
    
    fgerrL2[it] = sqrt(np.dot(fg, fg)/verts1.size())
    ff1L2[it] = sqrt(np.dot(ff1, ff1)/verts1.size())
    ff2L2[it] = sqrt(np.dot(ff2, ff2)/verts1.size())
    fgerrInf[it] = max(abs(fg))
    ff1Inf[it] = max(abs(ff1))
    ff2Inf[it] = max(abs(ff2))
    
     # dump to file
    mdb_coarse.write_file(output_coarse + '%i.vtk' % it)
    mdb_fine.write_file(output_fine + '%i.vtk' % it)

In [12]:
sio.savemat(
    matoutput1,
        {
            'GaussErrorL2': fgerrL2,
            'f1ErrorL2': ff1L2,
            'f2ErrorL2': ff2L2,
            'GaussErrorInf': fgerrInf,
            'f1ErrorInf': ff1Inf,
            'f2ErrorInf': ff2Inf
        }
)
sio.savemat(
    matoutput2,
    {
        'GaussErrorL2': cgerrL2,
        'f1ErrorL2': cf1L2,
        'f2ErrorL2': cf2L2,
        'GaussErrorInf': cgerrInf,
        'f1ErrorInf': cf1Inf,
        'f2ErrorInf': cf2Inf
    }
)